In [ ]:
# import pymysql
# from sqlalchemy import create_engine
# import sys
# from abc import ABC, abstractmethod
# from typing import *

import re

from prettytable import PrettyTable
import pandas as pd
pd.options.display.float_format = '{:.30f}'.format
import pprint

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### **0. 모듈의 상위 패키지를 파이썬 경로로 설정**

In [ ]:
import sys

In [ ]:
# sys.path
sys.path.append("C:\Pseudonymizer\pseudonymizer")
# sys.path.append("/Users/minjoo/codeStudy/psudonymizerTeam/pseudonymizer")
# /path/to/your/module

#### **1. MySQL Server 내 데이터베이스 테이블을 Python 스크립트와 연동하여 데이터 로드**

In [ ]:
from pseudonymizer.encryptionPseudonyms.pyMySQLQuery import PyMySQLQuery, ConnectMySQLserver

In [ ]:
# TEST CODE
queryObject = PyMySQLQuery(pw = "0123")
# queryObject = PyMySQLQuery(pw = "1234")

In [ ]:
queryObject.connectDatabase(
    serverIP = "localhost", 
    port_num = 3306, 
    user_name = "root", 
    database_name = "FINANCIALCONSUMER", 
    kr_encoder = "utf8")
SQL = "SELECT * FROM DATA_FINANCE"
queryObject.dataQueryLanguage(sql = SQL)

In [ ]:
# DATA_FINANCE_TUPLE = queryObject.executeQuery()
    # AttributeError: 'NoneType' object has no attribute 'execute'

In [ ]:
DATA_FINANCE_TABLE = queryObject.executeQueryAsDataFrame()

In [ ]:
type(DATA_FINANCE_TABLE)
DATA_FINANCE_TABLE.info()
DATA_FINANCE_TABLE.tail(3)

In [ ]:
queryObject2 = PyMySQLQuery(pw = "0123")
queryObject2.connectDatabase(
    serverIP = "localhost", 
    port_num = 3306, 
    user_name = "root", 
    database_name = "FINANCIALCONSUMER", 
    kr_encoder = "utf8")
SQL = "SELECT * FROM DATA_RETAIL"
queryObject2.dataQueryLanguage(sql = SQL)
DATA_RETAIL_TABLE = queryObject2.executeQueryAsDataFrame()

In [ ]:
type(DATA_RETAIL_TABLE)
DATA_RETAIL_TABLE.info()
DATA_RETAIL_TABLE.tail(3)

#### 2. **가명처리 절차 : 개인정보가 포함된 데이터를 비식별조치를 통해 가명정보로 변환**

In [ ]:
from pseudonymizer.pseudonymizer import Pseudonymizer

In [ ]:
from pseudonymizer.pseudonym import Pseudonym, PseudonymizerModule
    # ImportError: cannot import name 'Pseudonymizer' from 'pseudonymizer' (unknown location)

In [ ]:
help(Pseudonym)

In [ ]:
from pseudonymizer.pseudonymizers.maskingPseudonymizer import MaskingPseudonymizer
from pseudonymizer.pseudonymizers.charcategorization import CategorizationOfCharacter
from pseudonymizer.pseudonymizers.numcategorization import CategorizationOfNumeric
from pseudonymizer.pseudonymizers.columncategorization import CategorizationOfColumn
from pseudonymizer.pseudonymizers.microAggregation import MicroAggregation
from pseudonymizer.pseudonymizers.randomRoundingPseudonymizer import RandomRoundingPseudonymizer
from pseudonymizer.pseudonymizers.topandBottomCoding import TopandBottomCoding

In [ ]:
# 가명처리 기법 적용 객체 생성
pseudonym_object = Pseudonym(
    dataframe = DATA_FINANCE_TABLE,
    pseudonymizer_module = None
)
# 원본 데이터의 컬럼 정보 출력(.info())
# print(pseudonym_object)

In [ ]:
# 가명처리 기법 적용 객체 생성
pseudonym_object2 = Pseudonym(
    dataframe = DATA_RETAIL_TABLE,
    pseudonymizer_module = None
)

In [ ]:
# 속성값 조합을 이용한 동질집합 생성
pseudonym_object.categorizeEquivalentClass(
    attributes = ["NUM_CREDITCARD_ISSUANCES", "TF_LOAN", "TF_PENSION"]
)

In [ ]:
# 속성값 조합을 이용한 동질집합 생성
pseudonym_object2.categorizeEquivalentClass(
    attributes=["GENDER", "AGE"]
)

In [ ]:
# 동질집합 반환
pseudonym_object.countEquivalentClass()

```
# 속성값 조합을 이용한 동질집합 생성
pseudonym_object.categorizeEquivalentClass(
    attributes = ["NUM_CREDITCARD_ISSUANCES", "TF_LOAN", "TF_PENSION"]
)
```

In [ ]:
# 동질집합 반환
pseudonym_object2.countEquivalentClass()

---

##### **1) 가명처리기법 기능 모듈 로드: 연속형 변수 컬럼의 구간 범주화**

In [ ]:
# from pseudonymizer.pseudonymizers.columncategorization import CategorizationOfColumn
categorize_col = CategorizationOfColumn(numeric_type="bin", grouping_standard=["A", "B", "C"], right=True, ascending=False)

In [ ]:
categorize_col

##### **2.1) 가명처리기법 기능 모듈 로드: 문자형 변수 컬럼의 사용자 정의 범주화**

In [ ]:
# from pseudonymizer.pseudonymizers.charcategorization import CategorizationOfCharacter
catecorize_char_def = CategorizationOfCharacter(category_type = "user_definition", category_mapping = {"F": ["female"], "M": ["male"]})

In [ ]:
catecorize_char_def

##### **2.2)가명처리기법 기능 모듈 로드: 문자형 변수 컬럼의 날짜 범주화**

In [ ]:
# from pseudonymizer.pseudonymizers.charcategorization import CategorizationOfCharacter
categorize_char_date = CategorizationOfCharacter(category_type="date", category_mapping=None)

In [ ]:
categorize_char_date

##### **3.1) 가명처리기법 기능 모듈 로드: 특정 동질집합 특정 민감속성값의 연속형 변수 컬럼의 집계값 대체**

In [ ]:
# from pseudonymizer.pseudonymizers.microAggregation import MicroAggregation
micro_aggregation = MicroAggregation(calulate_type="average", quasi_identifier=(1, 'N', 'N'))

In [ ]:
micro_aggregation

##### **4.1) 가명처리기법 기능 모듈 로드: 연속형 변수 컬럼의 나이 구간 범주화**

In [ ]:
# from pseudonymizer.pseudonymizers.numcategorization import CategorizationOfNumeric
categorize_num = CategorizationOfNumeric(numeric_type = "age", grouping_standard = "5bin")

In [ ]:
categorize_num

##### **4.2) 가명처리기법 기능 모듈 로드: 연속형 변수 컬럼의 사용자 정의 구간 범주화**

In [ ]:
# from pseudonymizer.pseudonymizers.numcategorization import CategorizationOfNumeric
categorize_num_def = CategorizationOfNumeric(
    numeric_type="user_definition", category_mapping={"A": (0, 10000), "B": (10000, 50000), "C": (50000, 100000)})

In [ ]:
categorize_num_def

##### **5) 가명처리기법 기능 모듈 로드: 랜덤 라운딩**

In [ ]:
random_rounding = RandomRoundingPseudonymizer(rounding_type="round")

In [ ]:
random_rounding

##### **6.1) 가명처리기법 기능 모듈 로드: 모든 컬럼 마스킹 수행(비복원조치)**

In [ ]:
# from pseudonymizer.pseudonymizers.maskingPseudonymizer import MaskingPseudonymizer
masking = MaskingPseudonymizer(data_type="name", masking_domain=True, masking_part=None)

In [ ]:
masking

---

In [ ]:
adad = "sdsd"

In [ ]:
# 가명처리 기법 클래스 입력
pseudonym_object.addPseudonymizer(
    pseudonymizer = adad
)
# 입력받은 sdsd 기술은 가명처리 기법에 추가할 수 없습니다.

In [ ]:
# 적용 컬럼과 여러 가명처리 기법 정의
# 1)
pseudonym_object.addDictionary(
    column = "ZIP_CODE",
    pseudonymizers = [categorize_col]
)

In [ ]:
# 4)
pseudonym_object.addDictionary(
    column = "CREDIT_SCORE",
    pseudonymizers = [micro_aggregation]
)

In [ ]:
# 2.1)
pseudonym_object.addDictionary(
    column="GENDER",
    pseudonymizers=[categorize_char]
)

In [ ]:
# 2.2) 
pseudonym_object.addDictionary(
    column="AGE",
    pseudonymizers=[categorize_char_age]
)

In [ ]:
# 4.1)
pseudonym_object2.addDictionary(
    column="ZIP_CODE",
    pseudonymizers=[categorize_num_def]
)

In [ ]:
# 3.1)
pseudonym_object2.addDictionary(
    column="ZIP_CODE",
    pseudonymizers=[categorize_num_def]
)

In [ ]:
# 5)
pseudonym_object2.addDictionary(
    column="AMT_PURCHASES_BOOKS",
    pseudonymizers=[random_rounding]
)

In [ ]:
# 6.1)
pseudonym_object.addDictionary(
    column="NAME",
    pseudonymizers=[masking]
)

---

In [ ]:
# 가명처리 수행
pseudonym_object.pseudonymizeData()
pseudonym_object2.pseudonymizeData()

In [ ]:
# 가명정보 반환
pseudonym_data = pseudonym_object.getPseudonymizedDataframe()
pseudonym_data2 = pseudonym_object2.getPseudonymizedDataframe()

In [ ]:
display(pseudonym_data)
display(pseudonym_data2)

In [ ]:
pseudonym_object2._pseudonymDictionary

In [ ]:
pseudonym_object2.pseudonymizeData()

In [ ]:
pseudonym_object2.getPseudonymizedDataframe()

In [ ]:
pseudonym_object2.equivalent_class

In [ ]:
pseudonym_object2.pseudonymizeData()

In [ ]:
pseudonym_object2.getPseudonymizedDataframe()

In [ ]:
pseudonym_object = Pseudonym(
    dataframe = DATA_FINANCE_TABLE,
    pseudonymizer_module = None
)

In [ ]:
pseudonym_object.addDictionary(
    column="ZIP_CODE",
    pseudonymizers=[rd]
)

In [ ]:
pseudonym_object.pseudonymizeData()

In [ ]:
pseudonym_object.getPseudonymizedDataframe()

In [ ]:
pseudonym_object = Pseudonym(
    dataframe = DATA_FINANCE_TABLE,
    pseudonymizer_module = None
)

In [ ]:
pseudonym_object = Pseudonym(
    dataframe = DATA_RETAIL_TABLE,
    pseudonymizer_module = None
)

---

#### **3. 가명처리 절차 : 가명정보에 프라이버시 보호 모델 적용**

In [ ]:
from pseudonymizer.privacyPreservingModels import PrivacyPreservingModel

In [ ]:
help(PrivacyPreservingModel)